CORD

In [1]:
import os
import json

def normalize_bbox(bbox, width, height):
    bbox[0] = int(1000*(bbox[0] / width))
    bbox[1] = int(1000*(bbox[1] / height))
    bbox[2] = int(1000*(bbox[2] / width))
    bbox[3] = int(1000*(bbox[3] / height))

    return bbox


def generate_annotations(path: str):
    files = []

    for js in os.listdir(path):
        with open(path+js) as f:
            files.append(json.load(f))

    words = []
    boxes = []
    labels = []

    for js in files:
        text = []
        box = []
        label = []

        width, height = js['meta']['image_size']['width'], js['meta']['image_size']['height']

        for element in js['valid_line']:
            for word in element['words']:
                txt = word['text']

                x1 = word['quad']['x1']
                y1 = word['quad']['y1']
                x3 = word['quad']['x3']
                y3 = word['quad']['y3']

                bbox = [x1, y1, x3, y3]
                bbox = normalize_bbox(bbox, width, height)

                if len(txt) < 1:
                    continue
                if min(bbox) < 0 or max(bbox) > 1000:
                    continue
                if ((bbox[3] - bbox[1]) < 0) or ((bbox[2] - bbox[0]) < 0):
                    continue

                text.append(txt)
                box.append(bbox)
                label.append(element['category'])

        words.append(text)
        boxes.append(box)
        labels.append(label)
    
    return words, boxes, labels


In [2]:
train_path='Datasets/CORD/train/json/'
val_path = 'Datasets/CORD/dev/json/'
test_path = 'Datasets/CORD/test/json/'

words_train, boxes_train, labels_train = generate_annotations(train_path)
words_val, boxes_val, labels_val = generate_annotations(val_path)
words_test, boxes_test, labels_test = generate_annotations(test_path)

In [3]:
import pickle
with open('CORD_train.pkl', 'wb') as t:
    pickle.dump([words_train, labels_train, boxes_train], t)
with open('CORD_dev.pkl', 'wb') as t:
    pickle.dump([words_val, labels_val, boxes_val], t)
with open('CORD_test.pkl', 'wb') as t:
    pickle.dump([words_test, labels_test, boxes_test], t)

Finetuning LayoutLM

In [ ]:
from transformers import LayoutLMv2Processor
from PIL import Image

processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision='no-ocr')

image = Image.open()